In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report,r2_score
from sklearn.preprocessing import MinMaxScaler

""" Field Name	Order	Type (Format)	Description
Date	1	date (%Y-%m-%d)	Match Date (dd/mm/yy)                             -------> NO
HomeTeam	2	string (default)	Home Team
AwayTeam	3	string (default)	Away Team
FTHG	4	integer (default)	Full Time Home Team Goals
FTAG	5	integer (default)	Full Time Away Team Goals
FTR	6	string (default)	Full Time Result (H=Home Win, D=Draw, A=Away Win) -------> NO
HTHG	7	integer (default)	Half Time Home Team Goals
HTAG	8	integer (default)	Half Time Away Team Goals
HTR	9	string (default)	Half Time Result (H=Home Win, D=Draw, A=Away Win) -------> NO
HS	10	integer (default)	Home Team Shots
AS	11	integer (default)	Away Team Shots
HST	12	integer (default)	Home Team Shots on Target
AST	13	integer (default)	Away Team Shots on Target
HF	14	integer (default)	Home Team Fouls Committed
AF	15	integer (default)	Away Team Fouls Committed
HC	16	integer (default)	Home Team Corners
AC	17	integer (default)	Away Team Corners
HY	18	integer (default)	Home Team Yellow Cards
AY	19	integer (default)	Away Team Yellow Cards
HR	20	integer (default)	Home Team Red Cards
AR	21	integer (default)	Away Team Red Cards """


squadre =  {
                'Genoa' : 0,
                'Udinese' : 1,
                'Napoli'  : 2,
                'Empoli'  : 3,
                'Milan'   : 4,
                'Atalanta': 5,
                'Chievo'  : 6,
                'Juventus': 7,
                'Lazio'   : 8,
                'Spal'    : 9,
                'Parma'   : 10,
                'Cagliari': 11,
                'Sassuolo': 12,
                'Torino'  : 13,
                'Inter'   : 14,
                'Fiorentina': 15,
                'Bologna'   : 16,
                'Roma'      : 17,
                'Sampdoria' : 18,
                'Frosinone' : 19
            }

In [2]:
squadre =  {
                'Genoa' : 0,
                'Udinese' : 1,
                'Napoli'  : 2,
                'Empoli'  : 3,
                'Milan'   : 4,
                'Atalanta': 5,
                'Chievo'  : 6,
                'Juventus': 7,
                'Lazio'   : 8,
                'Spal'    : 9,
                'Parma'   : 10,
                'Cagliari': 11,
                'Sassuolo': 12,
                'Torino'  : 13,
                'Inter'   : 14,
                'Fiorentina': 15,
                'Bologna'   : 16,
                'Roma'      : 17,
                'Sampdoria' : 18,
                'Frosinone' : 19
            }

In [3]:
df = pd.read_csv("season-1819_csv.csv")

In [4]:
df.drop(['Date'], axis='columns', inplace=True) #Tolgo la colonna Date
df.drop(['FTR'], axis='columns', inplace=True) #Tolgo la colonna FTR
df.drop(['HTR'], axis='columns', inplace=True) #Tolgo la colonna HTR

In [5]:
df['HomeTeam'] = df['HomeTeam'].map(squadre)
df['AwayTeam'] = df['AwayTeam'].map(squadre)

In [187]:
df.drop(['FTHG_M'], axis='columns', inplace=True)

In [237]:
columns_home = ['FTHG', 'HTHG', 'HS', 'HST', 'HF', 'HC', 'HY', 'HR']

In [304]:
columns_away = df.columns.values
columns_away = np.delete(columns_away, np.argwhere(columns_away == 'HomeTeam'))
columns_away = np.delete(columns_away, np.argwhere(columns_away == 'AwayTeam'))
for i in range(len(columns_home)):
    index = np.argwhere(columns_away == columns_home[i])
    print(columns_home[i])
    print(index)
    columns_away = np.delete(columns_away, index)


FTHG
[[0]]
HTHG
[[1]]
HS
[[2]]
HST
[[3]]
HF
[[4]]
HC
[[5]]
HY
[[6]]
HR
[[7]]


In [305]:
columns_away

array(['FTAG', 'HTAG', 'AS', 'AST', 'AF', 'AC', 'AY', 'AR'], dtype=object)

In [303]:
columns_mean = []

In [306]:
#aggiungo le colonne per la media
for i in range (df.columns.size):
    if(df.columns[i] != 'HomeTeam' and df.columns[i] != 'AwayTeam'):
        columns_mean.append(df.columns[i] + '_M')
        df[df.columns[i] + '_M'] = 0
    

In [301]:
#tolgo le colonne per la media
for i in range (df.columns.size):
    if(df.columns[i].find('_M') != -1):
        df.drop(df.columns[i], axis='columns', inplace=True)
    

In [307]:
columns_mean

['FTHG_M',
 'FTAG_M',
 'HTHG_M',
 'HTAG_M',
 'HS_M',
 'AS_M',
 'HST_M',
 'AST_M',
 'HF_M',
 'AF_M',
 'HC_M',
 'AC_M',
 'HY_M',
 'AY_M',
 'HR_M',
 'AR_M']

In [235]:
df.columns[20].partition('_M')

'HTHG'

In [330]:
def mediaColonne(name_mean, is_home):
    for sq in range(20):
        column_mean = name_mean + '_M'
        if(is_home):
            mean_value = df.loc[df.HomeTeam == sq, name_mean].mean()
            df.loc[df.HomeTeam == sq, column_mean] = mean_value
        else:
            mean_value = df.loc[df.AwayTeam == sq, name_mean].mean()
            df.loc[df.AwayTeam == sq, column_mean] = mean_value

In [331]:
#faccio la media delle colonne della squadra di casa
for i in range(len(columns_home)):
        from_media_column = columns_home[i]
        mediaColonne(from_media_column, True)
        


In [332]:
#faccio la media delle colonne della squadra in trasferta
for i in range(len(columns_away)):
        from_media_column = columns_away[i]
        mediaColonne(from_media_column, False)
        

In [336]:
df.loc[df.HomeTeam == 0, columns_mean]

,FTHG_M,FTAG_M,HTHG_M,HTAG_M,HS_M,AS_M,HST_M,AST_M,HF_M,AF_M,HC_M,AC_M,HY_M,AY_M,HR_M,AR_M
13,1.214286,0.916667,0.714286,0.416667,10.642857,9.583333,5.285714,5.166667,13.642857,11.166667,5.571429,5.583333,2.571429,2.000000,0.142857,0.166667
33,1.214286,0.857143,0.714286,0.571429,10.642857,9.785714,5.285714,5.357143,13.642857,14.571429,5.571429,4.928571,2.571429,2.928571,0.142857,0.071429
52,1.214286,0.615385,0.714286,0.153846,10.642857,8.230769,5.285714,3.769231,13.642857,14.846154,5.571429,3.538462,2.571429,2.923077,0.142857,0.076923
74,1.214286,1.307692,0.714286,0.769231,10.642857,6.384615,5.285714,3.461538,13.642857,12.307692,5.571429,3.923077,2.571429,2.153846,0.142857,0.000000
93,1.214286,0.727273,0.714286,0.090909,10.642857,9.181818,5.285714,3.818182,13.642857,13.727273,5.571429,4.000000,2.571429,2.454545,0.142857,0.090909
111,1.214286,1.500000,0.714286,0.666667,10.642857,11.500000,5.285714,5.666667,13.642857,10.500000,5.571429,5.416667,2.571429,2.833333,0.142857,0.333333
125,1.214286,1.642857,0.714286,0.714286,10.642857,7.928571,5.285714,4.500000,13.642857,13.500000,5.571429,4.571429,2.571429,2.714286,0.142857,0.071429
145,1.214286,0.923077,0.714286,0.461538,10.642857,8.538462,5.285714,4.000000,13.642857,11.307692,5.571429,4.923077,2.571429,2.846154,0.142857,0.076923
162,1.214286,2.307692,0.714286,0.923077,10.642857,12.076923,5.285714,6.307692,13.642857,11.384615,5.571429,5.384615,2.571429,1.769231,0.142857,0.230769
182,1.214286,1.285714,0.714286,0.428571,10.642857,10.357143,5.285714,6.000000,13.642857,15.214286,5.571429,6.214286,2.571429,2.642857,0.142857,0.142857


In [528]:
####### Costruisco il modello
x_model_columns = ['HomeTeam', 'AwayTeam']
for i in range(len(columns_mean)):
    x_model_columns.append(columns_mean[i])
    
x_model = df.loc[21:, x_model_columns].values
y_model = df.loc[21:, 'FTHG'].values

In [529]:
y_model.shape

(238,)

In [530]:
from sklearn.tree import DecisionTreeClassifier

In [546]:
lr = DecisionTreeClassifier(max_depth=6)
x_train, x_test, y_train, y_test = train_test_split(x_model, y_model, test_size=0.20, shuffle=False)

In [622]:
lr.fit(x_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=6,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [623]:
pred = lr.predict(x_test)

In [624]:
print(pred)
print(y_test)

[3 2 2 2 1 1 1 0 0 3 1 0 0 2 1 0 2 0 1 2 1 0 2 0 2 3 1 0 2 5 0 2 0 1 0 0 3
 2 0 0 0 3 1 2 1 1 2 0]
[3 3 1 0 1 0 1 0 0 1 0 0 0 2 1 3 0 0 1 3 1 2 3 2 2 0 1 1 2 3 2 2 0 0 3 0 1
 1 2 3 3 1 3 0 1 1 3 2]


In [625]:
print(classification_report(y_true= y_test, y_pred= pred))

             precision    recall  f1-score   support

          0       0.41      0.50      0.45        14
          1       0.67      0.57      0.62        14
          2       0.31      0.44      0.36         9
          3       0.20      0.09      0.13        11
          5       0.00      0.00      0.00         0

avg / total       0.42      0.42      0.41        48



C:\Users\alfre\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [ ]:
# Random Forest

In [630]:
from sklearn.ensemble import RandomForestClassifier

In [703]:
forest = RandomForestClassifier(criterion='entropy' , n_estimators=11, n_jobs=2, random_state=3)

In [704]:
forest.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=11, n_jobs=2,
            oob_score=False, random_state=3, verbose=0, warm_start=False)

In [705]:
pred_forest = forest.predict(x_test)

In [706]:
print(classification_report(y_pred=pred_forest, y_true=y_test))

             precision    recall  f1-score   support

          0       0.69      0.64      0.67        14
          1       0.50      0.64      0.56        14
          2       0.25      0.33      0.29         9
          3       0.60      0.27      0.37        11

avg / total       0.53      0.50      0.50        48



In [723]:
from sklearn.metrics import precision_score

In [728]:
print(precision_score(y_true=y_test, y_pred=pred_forest, average='weighted'))

0.5321314102564103


In [ ]:
# BAgging

In [729]:
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier

In [815]:
bagging = BaggingClassifier(KNeighborsClassifier(n_neighbors=5), max_samples=0.5, max_features=0.5, random_state=3) 

In [816]:
bagging.fit(x_train, y_train)

BaggingClassifier(base_estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
         bootstrap=True, bootstrap_features=False, max_features=0.5,
         max_samples=0.5, n_estimators=10, n_jobs=1, oob_score=False,
         random_state=3, verbose=0, warm_start=False)

In [817]:
pred_bagging = bagging.predict(x_test)

In [818]:
print((precision_score(y_true=y_test, y_pred=pred_bagging, average='weighted')))

0.44682247899159666


In [819]:
print(classification_report(y_pred=pred_bagging, y_true=y_test))

             precision    recall  f1-score   support

          0       0.50      0.43      0.46        14
          1       0.53      0.64      0.58        14
          2       0.08      0.11      0.10         9
          3       0.57      0.36      0.44        11

avg / total       0.45      0.42      0.42        48



In [ ]:
# Confronto modelli

In [578]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import ARDRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier,VotingClassifier

In [626]:
clf1 = DecisionTreeClassifier(random_state=None, max_depth=6)
clf2 = RandomForestClassifier(n_estimators=50, criterion='gini',  n_jobs = -1,  random_state=None)
clf3 = GaussianNB()

In [627]:
eclf = VotingClassifier(estimators=[('rf1', clf1), ('rf2', clf2), ('gnb', clf3)], voting='soft')

In [628]:
for clf, label in zip([clf1, clf2, clf3, eclf], ['Linear Regression', 'Random Forest', 'Naive Bayes', 'Ensemble']):
    scores = cross_val_score(clf, x_test, y_test, cv = 5, scoring='accuracy')
    print(scores.std())
    print('Accuracy: %0.2f (+/- %0.2f)[%s]'  % (scores.mean(), scores.std(), label))

0.12678251737825585
Accuracy: 0.26 (+/- 0.13)[Linear Regression]
0.0977486493426121
Accuracy: 0.22 (+/- 0.10)[Random Forest]
0.1100618510850168
Accuracy: 0.30 (+/- 0.11)[Naive Bayes]


C:\Users\alfre\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\alfre\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\alfre\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\alfre\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but i

0.14379292095011198
Accuracy: 0.32 (+/- 0.14)[Ensemble]


C:\Users\alfre\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
